# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Co-option of complex molecular systems in bacterial and archaeal membranes	Talk	talk-1	JOBIM	05/07/2018	Marseille, France		
Co-option of complex molecular system in bacterial membranes	Talk	talk-2	AlPhy	02/02/2018	Montpellier, France		
Patterns of diversity and conservation among structural proteins of crAss-like phages	Poster	poster-1	Viruses of Microbes	18/07/2022	Guimaraes, Portugal		
Co-option of complex molecular system in bacterial membranes	Poster	poster-2	II Joint Congress on Evolutionary Biology	18/08/2018	Montpellier, France		
Co-option of complex molecular system in bacterial membranes	Poster	poster-3	Boris Ephrussi Day	14/05/2018	Paris, France		
Co-option of complex molecular system in bacterial membranes	Poster	poster-4	JOBIM	05/07/2017	Lille, France		
Co-option of complex molecular system in bacterial membranes	Poster	poster-5	Mathematical and Computational Evolutionary Biology	26/05/2017	Porquerolles Island, Fr

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0, dtype="string")
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Co-option of complex molecular systems in bact...,Talk,talk-1,JOBIM,05/07/2018,"Marseille, France",<NA>,<NA>
1,Co-option of complex molecular system in bacte...,Talk,talk-2,AlPhy,02/02/2018,"Montpellier, France",<NA>,<NA>
2,Patterns of diversity and conservation among s...,Poster,poster-1,Viruses of Microbes,18/07/2022,"Guimaraes, Portugal",<NA>,<NA>
3,Co-option of complex molecular system in bacte...,Poster,poster-2,II Joint Congress on Evolutionary Biology,18/08/2018,"Montpellier, France",<NA>,<NA>
4,Co-option of complex molecular system in bacte...,Poster,poster-3,Boris Ephrussi Day,14/05/2018,"Paris, France",<NA>,<NA>
5,Co-option of complex molecular system in bacte...,Poster,poster-4,JOBIM,05/07/2017,"Lille, France",<NA>,<NA>
6,Co-option of complex molecular system in bacte...,Poster,poster-5,Mathematical and Computational Evolutionary Bi...,26/05/2017,"Porquerolles Island, France",<NA>,<NA>
7,Disentangling the co-option of molecular funct...,Poster,poster-6,Young Researcher in Life Science,08/06/2016,"Paris, France",<NA>,<NA>
8,Macsyfinder v2 - Improved modelling and search...,Poster,poster-7,AlPhy - AEIM,23/01/2023,"Grenoble, France",<NA>,<NA>
9,Diversity of bacteriophages in the ancient hum...,Poster,poster-8,Viruses of Microbes,03/07/2023,"Tbilisi, Georgia",<NA>,<NA>


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"

    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
                
    md_filename = os.path.basename(md_filename)
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2016-06-08-poster-6.md 2018-05-14-poster-3.md 2022-07-18-poster-1.md
2016-poster-6.md       2018-07-05-talk-1.md   2022-poster-1.md
2017-05-26-poster-5.md 2018-08-18-poster-2.md 2023-01-23-Poster-7.md
2017-07-05-poster-4.md 2018-poster-2.md       2023-poster-10.md
2017-poster-4.md       2018-poster-3.md       2023-poster-7.md
2017-poster-5.md       2018-talk-1.md         2023-poster-8.md
2018-02-02-talk-2.md   2018-talk-2.md         2023-poster-9.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
